In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
datadir = os.path.join('..', 'data')
start_datetime = pd.to_datetime('2013-06-03 00:00:00')
end_datetime = pd.to_datetime('2013-06-10 00:00:00')

In [3]:
crs, fn = 'EPSG:2263', os.path.join(datadir, 'nybb', 'nybb.shp')
nyc = gpd.read_file(fn).set_geometry('geometry').to_crs(crs)

## Load, select & clean trip data

In [4]:
trip_data = pd.read_csv(os.path.join(datadir, 'trip_data.csv'))
trip_data.columns = trip_data.columns.str.strip()
for col in ('pickup_datetime', 'dropoff_datetime'):
    trip_data[col] = pd.to_datetime(trip_data[col])
    trip_data = (trip_data   # start <= datetime < end
                 .loc[trip_data[col] >= start_datetime]
                 .loc[trip_data[col] < end_datetime])
trip_data = trip_data.sort_values('pickup_datetime').dropna()

C:\Users\ccrea\AppData\Local\Temp\ipykernel_3660\660193016.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trip_data = pd.read_csv(os.path.join(datadir, 'trip_data.csv'))


In [5]:
# rate_code, store_and_fwd_flag = [1, 2, 3, 4, 5, 6, 99], ['Y', 'N']
rate_code_valid, rate_code_null = list(range(1, 7)), 99
msk_null = ~trip_data.rate_code.isin(rate_code_valid)
msk_flag = trip_data.store_and_fwd_flag.isna()
trip_data.loc[msk_null, 'rate_code'] = rate_code_null
trip_data.loc[msk_flag, 'store_and_fwd_flag'] = 'N'
trip_data['time_delta'] = trip_data.dropoff_datetime - trip_data.pickup_datetime
trip_data['trip_time'] = pd.to_timedelta(trip_data.trip_time_in_secs, unit = 's')
trip_data['trip_hours'] = trip_data.trip_time / pd.Timedelta(hours = 1)
trip_data['avg_speed'] = trip_data.trip_distance / trip_data.trip_hours
trip_data

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,time_delta,trip_time,trip_hours,avg_speed
10401493,D80D41E14AF130CB3B07FDFC3EC3B3EC,16F862053F48A8364FAF5A107BBA7B7A,CMT,1,N,2013-06-03 00:00:00,2013-06-03 00:14:36,1,876,4.1,-73.970673,40.748760,-73.962929,40.793980,0 days 00:14:36,0 days 00:14:36,0.243333,16.849315
10401496,51EA25825FE9F46A3F89898C8BB9DC97,1B66B6818C351F7DF958B1614D2256D2,CMT,1,N,2013-06-03 00:00:00,2013-06-03 00:03:48,1,226,0.4,-73.963966,40.807819,-73.959953,40.813587,0 days 00:03:48,0 days 00:03:46,0.062778,6.371681
10632109,B2EE6368DE7C92BEA45F0C85AD504AEE,9725BCE4DAA042EF689BCB0BCA05DD1B,CMT,1,N,2013-06-03 00:00:00,2013-06-03 00:04:35,1,274,0.1,-73.967560,40.763065,-73.965927,40.760956,0 days 00:04:35,0 days 00:04:34,0.076111,1.313869
10640581,C50DCB351C2A964A199EB023967029D7,88C0E51BB77C8FDA70D7F60F42D9A800,CMT,1,N,2013-06-03 00:00:00,2013-06-03 00:02:41,1,161,0.5,-73.993599,40.721279,-73.989281,40.726147,0 days 00:02:41,0 days 00:02:41,0.044722,11.180124
10646306,60091B1217A9E629BEA6AF080AAB9BCD,1E99E028E0C1DB2B1174AD8EED57301B,CMT,1,N,2013-06-03 00:00:00,2013-06-03 00:10:22,1,621,2.7,-73.979599,40.661190,-73.999435,40.685978,0 days 00:10:22,0 days 00:10:21,0.172500,15.652174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9322550,B0198EB0C55EE1DCF24D9108D6C080DC,9330C0495D04D66D5FBADEC892F9AE2C,CMT,2,N,2013-06-09 23:57:57,2013-06-09 23:58:14,1,17,0.0,-73.974960,40.744522,-73.974953,40.744537,0 days 00:00:17,0 days 00:00:17,0.004722,0.000000
9324188,D42076D7B45BCFBB8D544D765C2FB094,DC3A15804D7193B647BA71166CF161A8,CMT,1,N,2013-06-09 23:58:03,2013-06-09 23:59:36,1,92,0.4,-73.949142,40.785130,-73.944893,40.791264,0 days 00:01:33,0 days 00:01:32,0.025556,15.652174
9307696,56CF5E3DD6328847A51069D962EA293E,C00587622849F0BA2F250F7AAEFD74D6,CMT,1,N,2013-06-09 23:58:07,2013-06-09 23:59:45,1,98,0.3,-73.944946,40.834236,-73.948586,40.829330,0 days 00:01:38,0 days 00:01:38,0.027222,11.020408
9321404,D452FFF3E6A51D6235F0F886986776B9,A3E9537FA108A49E421C12019D2CC4C6,CMT,5,N,2013-06-09 23:59:09,2013-06-09 23:59:43,1,34,0.0,-74.035553,40.713402,-74.035553,40.713402,0 days 00:00:34,0 days 00:00:34,0.009444,0.000000


## Load, select & clean trip fare

In [6]:
trip_fare = pd.read_csv(os.path.join(datadir, 'trip_fare.csv'))
trip_fare.columns = trip_fare.columns.str.strip()
trip_fare['pickup_datetime'] = pd.to_datetime(trip_fare.pickup_datetime)
trip_fare = (trip_fare   # start <= datetime < end
             .loc[trip_fare.pickup_datetime >= start_datetime]
             .loc[trip_fare.pickup_datetime < end_datetime])
trip_fare['sum_charges'] = trip_fare[
    ['fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount']].sum(1)
trip_fare['tip_fare_ratio'] = trip_fare.tip_amount / trip_fare.fare_amount
trip_fare.sort_values('pickup_datetime', inplace = True)
trip_fare

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,sum_charges,tip_fare_ratio
4231944,EB9BE7C66F4F2886E7A20C7099E61814,C898129DD4BD6F617CFF39839F5AB969,VTS,2013-06-03 00:00:00,CSH,3.5,0.5,0.5,0.0,0.0,4.5,4.5,0.000000
4243270,150F059AE6614D5B9F243AB82F5359D9,88E6E03FDA0934EE7547CE282982F7FC,VTS,2013-06-03 00:00:00,CRD,25.5,0.5,0.5,3.0,0.0,29.5,29.5,0.117647
4243264,B2091923BB67756ED943F557F8402929,90EC6EDEA884074EA1E6F99BAACD0768,VTS,2013-06-03 00:00:00,CRD,23.0,0.5,0.5,5.0,0.0,29.0,29.0,0.217391
4243189,1AF6EC34D575B9CBBCACBA20A87E5CF3,B02558EF638DB42A82C3C5C9306FF68B,VTS,2013-06-03 00:00:00,CSH,21.5,0.5,0.5,0.0,0.0,22.5,22.5,0.000000
4223042,A56F5B8AC16B2B3A1B43938168AD4058,98D50A172EC96C5FCB98BF387EB651C6,VTS,2013-06-03 00:00:00,CRD,8.0,0.5,0.5,1.0,0.0,10.0,10.0,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10636445,1B5E728F7A41380C6BF635EAA6A816C5,5310EE41F9268E42222CACB4051EFAE1,CMT,2013-06-09 23:59:56,CRD,8.0,0.5,0.5,2.0,0.0,11.0,11.0,0.250000
9308513,5991156726C2A590BF4DB347973A4976,27BD53469A36001B7845388EE3429FCC,CMT,2013-06-09 23:59:57,CRD,6.0,0.5,0.5,2.1,0.0,9.1,9.1,0.350000
9301216,9032352777A85E4AC67BE6FB57B84A03,15CF8013E3B99AF97E3AD61C105DFCE4,CMT,2013-06-09 23:59:59,CRD,12.0,0.5,0.5,1.0,0.0,14.0,14.0,0.083333
9336259,D8AD4D1D5278894A0DF199C63098D411,23BD40520EEEC65CA754AE771A1F09E8,CMT,2013-06-09 23:59:59,CRD,11.0,0.5,0.5,2.4,0.0,14.4,14.4,0.218182


## Merge, clean, filter & deduplicate trip data+fare

In [7]:
keys = np.intersect1d(trip_data.columns, trip_fare.columns)
trip = gpd.GeoDataFrame(pd.merge(trip_data, trip_fare, 'inner', keys.tolist()))
for col in ('pickup', 'dropoff'):
    x, y = trip[f'{col}_longitude'], trip[f'{col}_latitude']
    trip[col] = gpd.points_from_xy(x = x, y = y, crs = 'EPSG:4326').to_crs(crs)
    trip.set_geometry(col, inplace = True)
    trip = gpd.sjoin(trip, nyc, predicate = 'within', how = 'left', rsuffix = col,
                     lsuffix = None if col == 'pickup' else 'pickup_')
trip.set_geometry('pickup', inplace = True)
trip['euclidean_distance'] = trip.pickup.distance(trip.dropoff) / 5280

In [8]:
dt, dlim = pd.Timedelta(minutes = 1), (.1, 30)
trip = (trip
        .loc[(trip.passenger_count > 0) & (trip.passenger_count < 10)]                   # 0 < passengers < 10
        .loc[trip.trip_time > dt]                                                        # time > 1 min
        .loc[trip.time_delta > dt]                                                       # time delta > 1 min
        .loc[(trip.trip_time - trip.time_delta).abs() < dt]                              # time equals time delta
        .loc[(trip.trip_distance > dlim[0]) & (trip.trip_distance < dlim[1])]            # .1 mi < distance < 30 mi
        .loc[(trip.euclidean_distance > dlim[0]) & (trip.euclidean_distance < dlim[1])]  # .1 mi < euclidean distance < 30 mi
        .loc[trip.trip_distance < 2 * trip.euclidean_distance]                           # distance < 2x euclidean distance
        .loc[(trip.avg_speed > 2) & (trip.avg_speed < 50)]                               # 2 < avg speed < 50
        .loc[trip.fare_amount < 200]                                                     # fare < $200
        .loc[trip.tip_fare_ratio < 5]                                                    # tip-to-fare ratio < 5
        .loc[(trip.sum_charges - trip.total_amount).abs() < 1e-2]                        # price equals charges
        .loc[~trip.index_pickup.isna() | ~trip.index_dropoff.isna()])                    # pickup or dropoff in NYC

In [9]:
by = ['medallion', 'hack_license', 'pickup', pd.Grouper(key = 'pickup_datetime', freq = 'min')]
trip = trip.groupby(by).first().sort_values('pickup_datetime').reset_index()
trip

,medallion,hack_license,pickup,pickup_datetime,vendor_id,rate_code,store_and_fwd_flag,dropoff_datetime,passenger_count,trip_time_in_secs,...,BoroName_pickup_,Shape_Leng_pickup_,Shape_Area_pickup_,dropoff,index_dropoff,BoroCode_dropoff,BoroName_dropoff,Shape_Leng_dropoff,Shape_Area_dropoff,euclidean_distance
0,6EE35E76FB48F9B79A337DDE55381126,16DA2E2A5B7CF561C21886460B636A31,POINT (9.98e+05 2.41e+05),2013-06-03 00:00:00,CMT,1,N,2013-06-03 00:04:25,1,206,...,Manhattan,357180.674063,6.363981e+08,POINT (9.96e+05 2.39e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.526704
1,9400FEF51A4DE705BD4578E3AB86A318,A0C7F56B74AB6A80B8B602B8F46262A8,POINT (9.92e+05 2.17e+05),2013-06-03 00:00:00,CMT,1,N,2013-06-03 00:05:23,4,310,...,Manhattan,357180.674063,6.363981e+08,POINT (9.96e+05 2.2e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.922924
2,2B131E11C91746501DB1E449ACFDDED5,2C24F168E6F6B8EC4A4D96E575C0EC40,POINT (1.02e+06 2.19e+05),2013-06-03 00:00:00,CMT,1,N,2013-06-03 00:15:53,1,911,...,Queens,896934.880411,3.045058e+09,POINT (9.99e+05 2.17e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,4.512526
3,D6B1A78F308FDE278D231C220F83D2A5,931B3AE4A8931DD58BA3E42766889BEC,POINT (9.93e+05 2.17e+05),2013-06-03 00:00:00,CMT,1,N,2013-06-03 00:16:43,1,944,...,Manhattan,357180.674063,6.363981e+08,POINT (1.01e+06 2.18e+05),4.0,4.0,Queens,896934.880411,3.045058e+09,2.873268
4,453B4DFA908932D761EF47FE919CDD72,A7B7AAEF95483779C50A7A0551EBD2E4,POINT (9.85e+05 2.01e+05),2013-06-03 00:00:00,CMT,1,N,2013-06-03 00:08:29,1,505,...,Manhattan,357180.674063,6.363981e+08,POINT (9.89e+05 2.03e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.952522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526788,FA48B4171DF5DAD8991253F81E2BF0FE,B00DC87B11CCEBDABBF86DD1D6E2CD07,POINT (9.85e+05 2.07e+05),2013-06-09 23:57:00,CMT,1,N,2013-06-09 23:58:30,1,74,...,Manhattan,357180.674063,6.363981e+08,POINT (9.85e+05 2.09e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.403466
1526789,1E268C16FEFBC4384CA90B84F8BDD060,1167EF28EEAC54FEDD1DBE1891B1855E,POINT (9.97e+05 2.23e+05),2013-06-09 23:57:00,CMT,1,N,2013-06-09 23:59:34,1,130,...,Manhattan,357180.674063,6.363981e+08,POINT (9.96e+05 2.2e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.517157
1526790,68D4B2FDB737B3E4449B8E8FCBEB0CC2,AF905278ACE8A86E989F0B79414571A9,POINT (9.9e+05 2.05e+05),2013-06-09 23:57:00,CMT,1,N,2013-06-09 23:59:59,1,138,...,Manhattan,357180.674063,6.363981e+08,POINT (9.89e+05 2.03e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.431326
1526791,D42076D7B45BCFBB8D544D765C2FB094,DC3A15804D7193B647BA71166CF161A8,POINT (9.98e+05 2.25e+05),2013-06-09 23:58:00,CMT,1,N,2013-06-09 23:59:36,1,92,...,Manhattan,357180.674063,6.363981e+08,POINT (1e+06 2.28e+05),3.0,1.0,Manhattan,357180.674063,6.363981e+08,0.478341


In [10]:
trip.to_parquet(os.path.join(datadir, 'trip_clean.parquet'))